# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_weather_df = pd.read_csv("../output_data/clean_cities.csv", usecols=["City", "Country", "Latitude", "Longitude", 
                                                                          "Temperature (F)", "Min Temperature (F)", 
                                                                          "Max Temperature (F)", "Wind (MPH)", 
                                                                          "Humidity (%)"])
city_weather_df

coords = city_weather_df[["Latitude", "Longitude"]]
humidity = city_weather_df["Humidity (%)"]



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

fig = gmaps.figure(zoom_level=3, center=(0,0))

heat_layer = gmaps.heatmap_layer(coords, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
temp_weather_df = city_weather_df.loc[(city_weather_df["Temperature (F)"] > 55) & \
                                             (city_weather_df["Wind (MPH)"] > 15)]
kite_flying_weather_df = temp_weather_df.dropna()

kite_flying_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Min Temperature (F),Max Temperature (F),Humidity (%),Wind (MPH)
5,rikitea,PF,-23.12,-134.97,72.84,72.84,72.84,80.0,16.15
12,altamont,US,42.21,-121.74,77.40,75.00,82.40,20.0,16.11
14,atuona,PF,-9.80,-139.03,78.76,78.76,78.76,76.0,16.26
25,port blair,IN,11.67,92.75,82.72,82.72,82.72,80.0,27.04
40,esperance,AU,-33.87,121.90,57.20,57.20,57.20,82.0,19.46
52,malinovoye ozero,RU,51.68,79.78,74.71,74.71,74.71,47.0,16.11
67,bambous virieux,MU,-20.34,57.76,69.42,69.01,69.80,64.0,17.22
77,kavaratti,IN,10.57,72.64,82.44,82.44,82.44,79.0,23.98
81,mahajanga,MG,-15.72,46.32,68.54,68.54,68.54,57.0,16.46
82,lamu,KE,-2.27,40.90,74.95,74.95,74.95,82.0,16.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = pd.DataFrame(columns=["Latitude", "Longitude", "City", "Country", "Hotel Name"])

hotel_df.head()

,Latitude,Longitude,City,Country,Hotel Name


In [9]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Grab the coordinates of the current row
target_coordinates = f"42.2117, -121.756"

# Set the other search parameters
target_search = "hotel"
target_radius = 2000
target_type = "hotel"

# set up a parameters dictionary
params = {
    'location': target_coordinates,
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key
    }

response = requests.get(url, params=params).json()
results = response['results']

response

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 42.21167699999999, 'lng': -121.7564106},
    'viewport': {'northeast': {'lat': 42.21268662989272, 'lng': -121.75506915},
     'southwest': {'lat': 42.20998697010728, 'lng': -121.75795015}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '5073da8c1b044ee67662d69f7ba004996a25c63e',
   'name': 'Best Western Plus Olympic Inn',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2160,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104643190143286197804">Best Western Olympic Inn</a>'],
     'photo_reference': 'CmRaAAAAD1i4b4vI2ScW5Nwmcfy9fYq6pP57Vs2p7H5blHTTE5ZonqeKB5qgaY39WxEMfkOG1cwxSeaeORO8WteVubOtsSN5ieinkcdESmVgkKhg4e-tafX88o-cALRkyxooXVjSEhBHLg3IFMmZEHq276hBpuPjGhQ7PWGGPtVqjC3WGmwvnxtDTn3iyg',
     'width': 3840}],
   'place_id': 'ChIJfyz58xHVyFQR6ratPtaZbHk',
   'plus_code': {'compound_code': '66

In [8]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Cycle over the rows in the kite_flying_weather
for index, row in kite_flying_weather_df.iterrows():

    print(f"Processing city: {kite_flying_weather_df['City']}")
    # Grab the coordinates of the current row
    target_coordinates = f"{row['Latitude']}, {row['Longitude']}"
    
    # Set the other search parameters
    target_search = "hotel"
    target_radius = 2000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
        }

    response = requests.get(url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        hotel_df.loc[index, "Latitude"] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Longitude"] = results[0]["geometry"]["location"]["lng"]
           
    except:
        pass

SyntaxError: EOL while scanning string literal (<ipython-input-8-8733e598a6ad>, line 6)

,Latitude,Longitude,City,Country,Hotel Name
5,-23.1277,-134.966,NaN,NaN,Pension Bianca & Benoit
12,42.2117,-121.756,NaN,NaN,Best Western Plus Olympic Inn
14,-9.80788,-139.03,NaN,NaN,Simplicité Marquises
25,11.6639,92.7557,NaN,NaN,Sinclairs Bayview Port Blair
40,-33.8529,121.9,NaN,NaN,The Jetty Resort
52,51.6721,79.7741,NaN,NaN,Sberbank
77,10.5706,72.6381,NaN,NaN,Bismillah Hotel And Lodging
81,-15.7236,46.3083,NaN,NaN,Lodge des Terres Blanches
82,-2.25558,40.8956,NaN,NaN,The Red Pepper House
101,24.8047,125.274,NaN,NaN,HOTEL LOCUS


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
